In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_1 = pd.read_csv('/content/drive/My Drive/Movie Recommender/TMDB 5000 Movie Dataset/tmdb_5000_credits.csv')
df_2 = pd.read_csv('/content/drive/My Drive/Movie Recommender/TMDB 5000 Movie Dataset/tmdb_5000_movies.csv')

In [ ]:
df_1

In [ ]:
df_2['overview']

** Bu kısımda tf - idf işlemi yapıldı.Kısaca:
- Stop Words yapılabilir.Kelimenin terim olup olmadığını saptamaya çalışır.
- Kelimenin toplam ağırlığını log ile hesaplar.Ağırlık değerine göre tf - idf oranı hesapnalıp kelimenin anlamlı hesaplamalarına bakılır.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [ ]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

In [ ]:
df_2['overview'] = df_2['overview'].fillna('')

In [ ]:
count_wm = countvectorizer.fit_transform(df_2['overview'])
tfidf_wm = tfidfvectorizer.fit_transform(df_2['overview'])

In [ ]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

In [ ]:
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

In [ ]:
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

CB filtering için Cosinüs benzerliğini kullandım.Farklı benzerlik bulma yöntemleri kullanılabilir.

In [ ]:
cosine_similarities = linear_kernel(tfidf_wm, tfidf_wm) 
results = {}
for idx, row in df_2.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], df_2['id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]

In [ ]:
def item(id):  
  return df_2.loc[df_2['id'] == id]['title'].tolist()[0].split(' - ')[0] 


In [ ]:
def recommend(item_id, num):

    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")



In [ ]:
recommend(item_id=11, num=5)